## Got this online kernel from Kaggle. It scores 0.63714

In [1]:
import pandas as pd
import numpy as np

dtypes = {'ad_id': np.float32, 'clicked': np.int8}

## Reads the training data and calculates the count, sum, and mean for each ad. Interesting: Completely strips the display_id here.

In [2]:
train = pd.read_csv("data/clicks_train.csv", usecols=['ad_id','clicked'], dtype=dtypes)

ad_likelihood = train.groupby('ad_id').clicked.agg(['count','sum','mean']).reset_index()
M = train.clicked.mean()
del train   # free up memory

## Good god what it that thing? I've asked on the forum why the 12. Otherwise, it looks like the equation just handles the case where there are no clicks on an ad despite it being shown. The ad will default to the global mean M.

In [7]:
ad_likelihood['likelihood'] = (ad_likelihood['sum'] + 12*M) / (12 + ad_likelihood['count'])
ad_likelihood.head()

,ad_id,count,sum,mean,likelihood
0,1.0,2,0.0,0.000000,0.165982
1,2.0,22,1.0,0.045455,0.097757
2,3.0,161,11.0,0.068323,0.077016
3,4.0,32,4.0,0.125000,0.143721
4,5.0,1,0.0,0.000000,0.178750


## Read the test data

In [ ]:
test = pd.read_csv("data/clicks_test.csv")

## Add the ad_likelihood to the test data

In [ ]:
test = test.merge(ad_likelihood, how='left')

In [8]:
test.head()

,display_id,ad_id,count,sum,mean,likelihood
32225159,23120126,145293,25321.0,4516.0,0.178350,0.178357
32225160,23120126,201649,51067.0,4074.0,0.079778,0.079804
32225161,23120126,292363,7602.0,589.0,0.077480,0.077663
32225158,23120126,57097,34294.0,1261.0,0.036770,0.036825
32225155,23120125,89725,19390.0,3991.0,0.205828,0.205820


## Replace any NaNs with the global mean

In [4]:
test.likelihood.fillna(M, inplace=True)

## Rearrange the table sorting the ads based on likelihood. This is a nice approach.

In [ ]:
test.sort_values(['display_id','likelihood'], inplace=True, ascending=False)

## Now just rearrange the dataframe so that the ads and listed after each display_id. Again, really elegant code.

In [ ]:
subm = test.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()

In [6]:
subm.head()

,display_id,ad_id
0,16874594,170392 172888 162754 150083 66758 180797
1,16874595,8846 143982 30609
2,16874596,289915 11430 289122 132820 57197 153260 173005...
3,16874597,305790 285834 143981 182039 155945 180965 3088...
4,16874598,145937 335632 67292 250082


In [5]:
subm.to_csv("data/subm.csv", index=False)